# Building a Question-Answering System with Hugging Face Transformers

In [1]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering, pipeline

id = 'deepset/minilm-uncased-squad2'
tokenizer = AutoTokenizer.from_pretrained(id)
model = TFAutoModelForQuestionAnswering.from_pretrained(id, from_pt=True)
pipe = pipeline('question-answering', model=model, tokenizer=tokenizer)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [2]:
question = 'What does NLP stand for?'

context = 'Natural Language Processing, or NLP, encompasses a variety of activities, \
           including text classification, keyword and topic extraction, text \
           summarization, and language translation. The accuracy of NLP models \
           has improved in recent years for a variety of reasons, not the least \
           of which are newer and better ways of converting words and sentences into \
           dense vector representations that incorporate word context, and a \
           relatively new neural-network architecture called the transformer that \
           can zero in on the most meaningful words and even differentiate \
           between different meanings of the same word.'

output = pipe(question=question, context=context)
print(output)

{'score': 0.9793134331703186, 'start': 0, 'end': 27, 'answer': 'Natural Language Processing'}


In [3]:
question = 'When was TensorFlow released?'

context = 'Machine learning isn\'t hard when you have a properly engineered dataset \
           to work with. The reason it\'s not hard is libraries such as Scikit-learn \
           and ML.NET, which reduce complex learning algorithms to a few lines of \
           code. Deep learning isn’t difficult, either, thanks to libraries such as \
           the Microsoft Cognitive Toolkit (CNTK), Theano, and PyTorch. But the \
           library that most of the world has settled on for building neural networks \
           is TensorFlow, an open-source framework created by Google that was \
           released under the Apache License 2.0 in 2015.'

output = pipe(question=question, context=context)
print(output['answer'])

2015


In [4]:
question = 'Is Keras part of TensorFlow?'

context =  'The learning curve for TensorFlow is rather steep. Another library \
            named Keras provides a simplified Python interface to TensorFlow \
            and has emerged as the Scikit of deep learning. Keras is all about \
            neural networks. It began life as a stand-alone project in 2015 \
            but was integrated into TensorFlow in 2019. Any code that you write \
            using TensorFlow’s built-in Keras module ultimately executes in \
            (and is optimized for) TensorFlow. Even Google recommends using the \
            Keras API.'

output = pipe(question=question, context=context)
print(output['answer'])

integrated into TensorFlow in 2019


In [5]:
question = 'Is it better to use Keras or TensorFlow to build neural networks?'
output = pipe(question=question, context=context)
print(output['answer'])

Keras


## Load contexts and implement a retriever

In [6]:
import pandas as pd

data = pd.read_csv('Data/passages.csv', header=None)
pd.set_option('display.max_colwidth', None)
data.head()

,0
0,"Machine learning isn’t hard when you have a properly engineered dataset to work with. The reason it’s not hard is libraries such as Scikit-learn and ML.NET, which reduce complex learning algorithms to a few lines of code. Deep learning isn’t difficult, either, thanks to libraries such as the Microsoft Cognitive Toolkit (CNTK), Theano, and PyTorch. But the library that most of the world has settled on for building neural networks is TensorFlow, an open-source framework created by Google that was released under the Apache License 2.0 in 2015."
1,"TensorFlow isn’t limited to building neural networks. It is a framework for performing fast mathematical operations at scale using tensors, which are generalized versions of arrays. Tensors can represent scalar values (0-dimensional tensors), vectors (1D tensors), matrices (2D tensors), and so on. A neural network is basically a workflow for transforming tensors. The 3-layer perceptron featured in the previous chapter takes a 1D tensor containing two values as input, transforms it into a 1D tensor containing three values, and produces a 0D tensor as output. TensorFlow lets you define directed graphs that in turn define how tensors are computed. And unlike Scikit, it supports GPUs."
2,The learning curve for TensorFlow is rather steep. Another library named Keras provides a simplified Python interface to TensorFlow and has emerged as the Scikit of deep learning. Keras is all about neural networks. It began life as a stand-alone project in 2015 but was integrated into TensorFlow in 2019. Any code that you write using TensorFlow’s built-in Keras module ultimately executes in (and is optimized for) TensorFlow. Even Google recommends using the Keras API.
3,"Keras offers two APIs for building neural networks: a sequential API and a functional API. The former is simpler and is sufficient for most neural networks. The latter is useful in more advanced scenarios such as networks with multiple outputs – for example, a classification output and a regression output, which is common in neural networks that perform object detection – or shared layers."
4,"It’s not difficult to use Scikit-learn to build machine-learning models that analyze text for sentiment, identify spam, and classify text in other ways. But today, state-of-the-art text classification is most often performed with neural networks. You already know how to build neural networks that accept numbers and images as input. Let’s build on that to learn how to construct deep-learning models that process text – a segment of deep learning known as natural language processing, or NLP for short."


In [7]:
from transformers import TFAutoModel

bert_id = 'sebastian-hofstaetter/distilbert-dot-margin_mse-T2-msmarco'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_id) 
bert_model = TFAutoModel.from_pretrained(bert_id, from_pt=True)

All PyTorch model weights were used when initializing TFDistilBertModel.

All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [8]:
def vectorize_text(text):
    tokenized_text = bert_tokenizer(text, return_tensors='tf')
    vectorized_text = bert_model(tokenized_text)[0][:, 0, :][0]
    return vectorized_text

contexts = data[0]
vectorized_contexts = contexts.apply(vectorize_text)

In [9]:
import numpy as np

def get_best_contexts(query, contexts, max_matches=3):
    scores = pd.Series(dtype='object')
    tokenized_query = bert_tokenizer(query, return_tensors='tf')
    vectorized_query = bert_model(tokenized_query)[0][:, 0, :][0]
    
    for idx, item in contexts.iteritems():
        score = np.dot(vectorized_query, item)
        scores = scores.append(pd.Series(score), ignore_index=True)

    sorted_scores = scores.sort_values(ascending=False)[:max_matches]
    return list(sorted_scores.index)

In [10]:
question = 'How many versions of YOLO are there?'
indexes = get_best_contexts(question, vectorized_contexts)

for idx in indexes:
    print(f'{contexts[idx]}\n')

There are five versions of YOLO referred to as YOLOv1 through YOLOv5. There are also variations such as PP-YOLO and YOLO9000. YOLOv3 processes the image using a 13x13 grid, a 26x26 grid, and a 52x52 grid so it can detect objects large and small. It borrows the anchor-box concept from Faster R-CNN to predict nine bounding boxes per cell. YOLO’s primary weakness is that it has difficulty detecting small objects that are close together. This is a consequence of the fact that a given cell can only predict one class. 

While the R-CNN family of object-detection systems delivers unparalleled accuracy, it leaves something to be desired when it comes to real-time object detection of the type required by, say, self-driving cars. A paper entitled “You Only Look Once: Unified, Real-Time Object Detection” published in 2015 proposed an alternative to R-CNNs known as YOLO that revolutionized the way data scientists think about object detection. 

Until a few short years ago, most NMT models, includi

In [11]:
for idx in indexes:
    output = pipe(question=question, context=contexts[idx], handle_impossible_answer=True)
    
    if output['start'] != output['end']:
        print(output)

{'score': 0.4768390357494354, 'start': 10, 'end': 14, 'answer': 'five'}


In [12]:
def show_answers(question, contexts, vectorized_contexts):
    indexes = get_best_contexts(question, vectorized_contexts)
    
    for idx in indexes:
        output = pipe(question=question, context=contexts[idx], handle_impossible_answer=True)

        if output['start'] != output['end']:
            print(output)

question = 'What type of neural network supports instance segmentation?'
show_answers(question, contexts, vectorized_contexts)

{'score': 0.8453524112701416, 'start': 83, 'end': 94, 'answer': 'Mask R-CNNs'}


In [13]:
question = 'What is YOLO\'s primary weakness?'
show_answers(question, contexts, vectorized_contexts)

{'score': 0.5289613604545593, 'start': 371, 'end': 436, 'answer': 'it has difficulty detecting small objects that are close together'}


In [14]:
question = 'Is TensorFlow difficult to learn?'
show_answers(question, contexts, vectorized_contexts)

{'score': 0.12999455630779266, 'start': 436, 'end': 446, 'answer': 'TensorFlow'}
{'score': 0.2692341208457947, 'start': 0, 'end': 49, 'answer': 'The learning curve for TensorFlow is rather steep'}
